In [2]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import f1_score, roc_auc_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora, models, matutils
import numpy as np
import ast

In [18]:
## Import data

df = pd.read_csv('datasets/11_features.csv')

Index(['Unnamed: 0', 'claim_id', 'claim_label', 'document_bow_19.5',
       'document_tfidf_19.5', 'document_lsi_19.5_20', 'document_lda_19.5_20',
       'document_rp_19.5_20', 'document_lsi_19.5_40', 'document_lda_19.5_40',
       ...
       'claim_rp_0.06_20', 'claim_lsi_0.06_30', 'claim_lda_0.06_30',
       'claim_rp_0.06_30', 'claim_lsi_0.06_40', 'claim_lda_0.06_40',
       'claim_rp_0.06_40', 'claim_lsi_0.06_50', 'claim_lda_0.06_50',
       'claim_rp_0.06_50'],
      dtype='object', length=2057)

In [10]:
## We create 4 list of features
## - 2 lists for claim features
## - 2 lists for document features
## - for claim and documents, one list for 
## base models features (tfidf, bow) and 
## one for topic models features

feature_categories_claim_topic = []
feature_categories_document_topic = []
feature_categories_claim_count = []
feature_categories_document_count = []
for nb_word in sorted(list(np.arange(2000,20000,2000)),reverse=True):
    for f in ['bow','tfidf']:
        feature_categories_document_count.append('document_'+f+'_'+str(nb_word/1000))
    for nb_topic in np.arange(100,301,40):
        for f in ['lsi','lda','rp']:
            feature_categories_document_topic.append('document_'+f+'_'+str(nb_word/1000)+'_'+str(nb_topic))
            
for nb_word in sorted(list(np.arange(90,186,10)),reverse=True):
    for f in ['bow','tfidf']:
        feature_categories_claim_count.append('claim_'+f+'_'+str(nb_word/1000))
    for nb_topic in np.arange(10,51,10):
        for f in ['lsi','lda','rp']:
            feature_categories_claim_topic.append('claim_'+f+'_'+str(nb_word/1000)+'_'+str(nb_topic))

In [ ]:
## We eval strings as lists of features

for category in feature_categories_claim_topic:
    df[category] = df[category].apply(lambda st: ast.literal_eval(st))
for category in feature_categories_document_topic:
    df[category] = df[category].apply(lambda st: ast.literal_eval(st))
for category in feature_categories_claim_count:
    df[category] = df[category].apply(lambda st: ast.literal_eval(st))
for category in feature_categories_document_count:
    df[category] = df[category].apply(lambda st: ast.literal_eval(st))

In [15]:
## The problem is that we have lists inside lists

for category in feature_categories_claim_topic:
    tab = category.split('_')
    nb_dimensions = int(category.split('_')[-1])
    test_df = matutils.corpus2dense(df[category],nb_dimensions).transpose()
    test_df_2 = [str(list(x)) for x in test_df]
    test_df_3 = pd.Series(test_df_2).apply(lambda x: ast.literal_eval(x))
    test_df_3.index = df[category].index
    df[category] = test_df_3
for category in feature_categories_document_topic:
    nb_dimensions = int(category.split('_')[-1])
    test_df = matutils.corpus2dense(df[category],nb_dimensions).transpose()
    test_df_2 = [str(list(x)) for x in test_df]
    test_df_3 = pd.Series(test_df_2).apply(lambda x: ast.literal_eval(x))
    test_df_3.index = df[category].index
    df[category] = test_df_3

for category in feature_categories_claim_count:
    tab = category.split('_')
    nb_dimensions = int(float(category.split('_')[-1])*1000)
    test_df = matutils.corpus2dense(df[category],nb_dimensions).transpose()
    test_df_2 = [str(list(x)) for x in test_df]
    test_df_3 = pd.Series(test_df_2).apply(lambda x: ast.literal_eval(x))
    test_df_3.index = df[category].index
    df[category] = test_df_3
for category in feature_categories_document_count:
    nb_dimensions = int(float(category.split('_')[-1])*1000)
    test_df = matutils.corpus2dense(df[category],nb_dimensions).transpose()
    test_df_2 = [str(list(x)) for x in test_df]
    test_df_3 = pd.Series(test_df_2).apply(lambda x: ast.literal_eval(x))
    test_df_3.index = df[category].index
    df[category] = test_df_3

/home/ymerej/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ymerej/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/ymerej/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [16]:
## We create configurations for gridsearch optimisation with 10 fold cross-validation

skf = StratifiedKFold(n_splits=10)
params_feature_category_selection = {'name': 'logistic regression', 'model': LogisticRegression(), 'param_grid':[
    {'penalty':['l2'], 'dual':[True,False], 'C': [1, 10, 100, 1000],'fit_intercept':[False]},
    {'penalty':['l2'], 'dual':[True,False], 'C': [1, 10, 100, 1000],'fit_intercept':[True], 
     'intercept_scaling':[1,10,100,1000]},
    {'penalty':['l1'], 'dual':[False], 'C': [1, 10, 100, 1000],'fit_intercept':[False]},
    {'penalty':['l1'], 'dual':[False], 'C': [1, 10, 100, 1000],'fit_intercept':[True], 
     'intercept_scaling':[1,10,100,1000]}
]}

## Here we create dictionaries that will contain results from training and testing models

grid_claim_category_selection_topic = {name: None for name in feature_categories_claim_topic}
grid_document_category_selection_topic = {name: None for name in feature_categories_document_topic}

grid_claim_category_selection_count = {name: None for name in feature_categories_claim_count}
grid_document_category_selection_count = {name: None for name in feature_categories_document_count}

In [19]:
## For each category of feature, we fit models using gridsearch optimisation
## and ten fold cross-validation and we get the results

for category_name in feature_categories_claim_topic:
    gs = GridSearchCV(params_feature_category_selection['model'], 
                      params_feature_category_selection['param_grid'], cv = skf, scoring={'F1':'f1'}, refit=False)
    grid_claim_category_selection_topic[category_name] = gs.fit(np.array(list(df[category_name].as_matrix())),
                                                              df['claim_label'].apply(lambda x: 1 if x else 0).as_matrix())
    
for category_name in feature_categories_document_topic:
    gs = GridSearchCV(params_feature_category_selection['model'], 
                      params_feature_category_selection['param_grid'], cv = skf, scoring={'F1':'f1'}, refit=False)
    grid_document_category_selection_topic[category_name] = gs.fit(np.array(list(df[category_name].as_matrix())),
                                                              df['claim_label'].apply(lambda x: 1 if x else 0).as_matrix())
    
for category_name in feature_categories_claim_count:
    gs = GridSearchCV(params_feature_category_selection['model'], 
                      params_feature_category_selection['param_grid'], cv = skf, scoring={'F1':'f1'}, refit=False)
    grid_claim_category_selection_count[category_name] = gs.fit(np.array(list(df[category_name].as_matrix())),
                                                              df['claim_label'].apply(lambda x: 1 if x else 0).as_matrix())
    
for category_name in feature_categories_document_count:
    gs = GridSearchCV(params_feature_category_selection['model'], 
                      params_feature_category_selection['param_grid'], cv = skf, scoring={'F1':'f1'}, refit=False)
    grid_document_category_selection_count[category_name] = gs.fit(np.array(list(df[category_name].as_matrix())),
                                                              df['claim_label'].apply(lambda x: 1 if x else 0).as_matrix())

/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/m

/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/m

/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/m

/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/m

/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/m

...

/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/m

/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/m

/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/m

/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/m

/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ymerej/anaconda3/lib/python3.6/site-packages/sklearn/m

In [20]:
## For each category of features and for each machine learning technique
## we keep the configuration from gridsearch with the best mean F1 score across folds

grid_claim_category_selection_topic_2 = {k:np.max(v.cv_results_['mean_test_F1']) \
                                   for k,v in grid_claim_category_selection_topic.items()}

grid_document_category_selection_topic_2 = {k:np.max(v.cv_results_['mean_test_F1']) \
                                   for k,v in grid_document_category_selection_topic.items()}

grid_claim_category_selection_count_2 = {k:np.max(v.cv_results_['mean_test_F1']) \
                                   for k,v in grid_claim_category_selection_count.items()}

grid_document_category_selection_count_2 = {k:np.max(v.cv_results_['mean_test_F1']) \
                                   for k,v in grid_document_category_selection_count.items()}
                                

In [21]:
## We decompose the name of the feature category 
## into size of lexicon, (nb of topics)?, topic or base model technique

grid_claim_category_selection_topic_3 = [(int(float(k.split('_')[2])*1000),int(k.split('_')[3]),k.split('_')[1],v) \
                                   for k, v in grid_claim_category_selection_topic_2.items()]
grid_document_category_selection_topic_3 = [(int(float(k.split('_')[2])*1000),int(k.split('_')[3]),k.split('_')[1],v) \
                                      for k, v in grid_document_category_selection_topic_2.items()]

grid_claim_category_selection_count_3 = [(int(float(k.split('_')[2])*1000),k.split('_')[1],v) \
                                   for k, v in grid_claim_category_selection_count_2.items()]
grid_document_category_selection_count_3 = [(int(float(k.split('_')[2])*1000),k.split('_')[1],v) \
                                      for k, v in grid_document_category_selection_count_2.items()]

In [22]:
## We create multiindex

tuples_claims_topic = [x[:3] for x in grid_claim_category_selection_topic_3]
tuples_documents_topic = [x[:3] for x in grid_document_category_selection_topic_3]

index_claims_topic = pd.MultiIndex.from_tuples(tuples_claims_topic, 3, ['nb_word','nb_topic','representation'])
index_documents_topic = pd.MultiIndex.from_tuples(tuples_documents_topic, 3, ['nb_word','nb_topic','representation'])

tuples_claims_count = [x[:2] for x in grid_claim_category_selection_count_3]
tuples_documents_count = [x[:2] for x in grid_document_category_selection_count_3]

index_claims_count = pd.MultiIndex.from_tuples(tuples_claims_count, 2, ['nb_word','representation'])
index_documents_count = pd.MultiIndex.from_tuples(tuples_documents_count, 2, ['nb_word','representation'])

In [23]:
## we create series from the dictionaries containing results and multiindex

grid_claim_category_selection_topic_4 = [x[3] for x in grid_claim_category_selection_topic_3]
grid_document_category_selection_topic_4 = [x[3] for x in grid_document_category_selection_topic_3]

se_claim_topic = pd.Series(grid_claim_category_selection_topic_4, index=index_claims_topic)
se_document_topic = pd.Series(grid_document_category_selection_topic_4, index=index_documents_topic)

grid_claim_category_selection_count_4 = [x[2] for x in grid_claim_category_selection_count_3]
grid_document_category_selection_count_4 = [x[2] for x in grid_document_category_selection_count_3]

se_claim_count = pd.Series(grid_claim_category_selection_count_4, index=index_claims_count)
se_document_count = pd.Series(grid_document_category_selection_count_4, index=index_documents_count)

#df_category_selection

In [41]:
## We look at the F-measure score for base features on claims

se_claim_count.apply(lambda x: int(x*100)/100).unstack().transpose()

nb_word,90,100,110,120,130,140,150,160,170,180
representation,,,,,,,,,,
bow,0.61,0.57,0.57,0.56,0.58,0.58,0.58,0.60,0.61,0.6
tfidf,0.62,0.60,0.60,0.59,0.60,0.59,0.58,0.61,0.60,0.6


In [47]:
## We look at the F-measure score for base features on documents

se_document_count.apply(lambda x: int(x*100)/100).unstack().transpose()

nb_word,2000,4000,6000,8000,10000,12000,14000,16000,18000
representation,,,,,,,,,
bow,0.53,0.48,0.48,0.48,0.47,0.46,0.48,0.47,0.48
tfidf,0.57,0.50,0.53,0.46,0.53,0.46,0.60,0.47,0.47


In [48]:
## We look at the F-measure score for topic features on claims

se_claim_topic.apply(lambda x: int(x*100)/100).unstack().unstack().transpose()

nb_word                   90    100   110   120   130   140   150   160   170  \
representation nb_topic                                                         
lda            10        0.59  0.67  0.64  0.57  0.55  0.59  0.53  0.58  0.51   
               20        0.62  0.62  0.58  0.62  0.55  0.53  0.60  0.59  0.61   
               30        0.62  0.65  0.62  0.55  0.50  0.59  0.61  0.59  0.63   
               40        0.56  0.62  0.52  0.59  0.62  0.53  0.52  0.54  0.61   
               50        0.64  0.52  0.62  0.53  0.58  0.53  0.49  0.51  0.54   
lsi            10        0.64  0.61  0.61  0.60  0.61  0.61  0.61  0.64  0.61   
               20        0.65  0.63  0.64  0.64  0.64  0.64  0.64  0.62  0.63   
               30        0.63  0.63  0.65  0.62  0.61  0.63  0.63  0.62  0.60   
               40        0.63  0.62  0.63  0.63  0.62  0.63  0.62  0.60  0.59   
               50        0.63  0.62  0.64  0.62  0.63  0.61  0.60  0.59  0.59   
rp             10        0.58  0.55  0.55  0.57  0.50  0.51  0.52  0.52  0.60   
               20        0.58  0.50  0.59  0.56  0.53  0.61  0.58  0.60  0.60   
               30        0.53  0.56  0.54  0.63  0.55  0.50  0.55  0.52  0.56   
               40        0.61  0.44  0.59  0.57  0.55  0.58  0.54  0.57  0.55   
               50        0.57  0.59  0.57  0.66  0.58  0.60  0.56  0.53  0.62   

nb_word                   180  
representation nb_topic        
lda            10        0.59  
               20        0.60  
               30        0.65  
               40        0.61  
               50        0.57  
lsi            10        0.62  
               20        0.61  
               30        0.62  
               40        0.60  
               50        0.59  
rp             10        0.50  
               20        0.56  
               30        0.53  
               40        0.50  
               50        0.54

In [49]:
## We look at the F-measure score for base features on documents

se_document_topic.apply(lambda x: int(x*100)/100).unstack().unstack().transpose()

nb_word                  2000   4000   6000   8000   10000  12000  14000  \
representation nb_topic                                                    
lda            100        0.64   0.57   0.54   0.55   0.57   0.48   0.55   
               140        0.55   0.47   0.49   0.47   0.53   0.58   0.44   
               180        0.60   0.48   0.53   0.56   0.48   0.53   0.57   
               220        0.54   0.55   0.53   0.62   0.60   0.58   0.53   
               260        0.49   0.60   0.46   0.47   0.58   0.52   0.55   
               300        0.46   0.52   0.54   0.58   0.66   0.55   0.53   
lsi            100        0.51   0.53   0.60   0.49   0.52   0.54   0.52   
               140        0.53   0.52   0.52   0.55   0.53   0.54   0.49   
               180        0.51   0.53   0.66   0.59   0.51   0.54   0.55   
               220        0.51   0.63   0.59   0.53   0.51   0.54   0.53   
               260        0.52   0.50   0.51   0.46   0.54   0.60   0.52   
               300        0.51   0.53   0.53   0.46   0.54   0.54   0.53   
rp             100        0.51   0.54   0.53   0.54   0.53   0.51   0.50   
               140        0.51   0.54   0.50   0.51   0.54   0.50   0.48   
               180        0.50   0.49   0.52   0.53   0.46   0.53   0.52   
               220        0.51   0.47   0.50   0.54   0.50   0.54   0.44   
               260        0.53   0.46   0.48   0.53   0.56   0.53   0.50   
               300        0.52   0.46   0.53   0.48   0.52   0.47   0.46   

nb_word                  16000  18000  
representation nb_topic                
lda            100        0.49   0.46  
               140        0.56   0.50  
               180        0.47   0.58  
               220        0.58   0.57  
               260        0.47   0.52  
               300        0.47   0.47  
lsi            100        0.55   0.53  
               140        0.51   0.49  
               180        0.50   0.52  
               220        0.50   0.48  
               260        0.53   0.48  
               300        0.50   0.52  
rp             100        0.54   0.52  
               140        0.51   0.51  
               180        0.55   0.52  
               220        0.51   0.52  
               260        0.54   0.50  
               300        0.54   0.51

In [95]:
## The two topic features on claim with best f-measure scores
se_claim.sort_values(ascending=False).head(2)

nb_word  nb_topic  representation
60       10        lsi               0.645552
         50        lda               0.644990
dtype: float64

In [94]:
## The two topic features on documents with best f-measure scores
se_document.sort_values(ascending=False).head(2)

nb_word  nb_topic  representation
10000    150       lda               0.596531
                   rp                0.589153
dtype: float64

In [22]:
## We select the categories of features with best f-measure scores for both claims and documents
## We configure gridsearch with a more complex model


selected_categories = ['claim_lda_0.1_10', 'document_lda_10.0_300', 'document_lsi_6.0_180']
params = [
    {'name': 'svc', 'model': svm.SVC(), 'param_grid':[
        {'kernel':['linear'], 'C': [1,10,100,1000]},
        {'kernel':['poly'], 'degree':[2,3], 'gamma': [0.001,0.0001], 'C': [1,10,100,1000]},
        {'kernel':['rbf','sigmoid'], 'gamma': [0.001, 0.0001], 'C': [1,10,100,1000]}
    ]}
    #{'name': 'naive bayes', 'model': MultinomialNB(), 'param_grid':{'alpha':[1, 10, 100, 1000]}},
    #{'name': 'random forest', 'model': RandomForestClassifier(), 
     #'param_grid':{'n_estimators':[10,100], 'criterion':['gini','entropy'],
     #             'max_features':['sqrt','log2'], 'max_depth':[10,100], 
     #              'min_samples_split':[0.01, 0.1], 'min_samples_leaf':[0.01,0.1],
     #             'min_weight_fraction_leaf': [0.01], 'max_leaf_nodes': [10]}}#,
    #{'name': 'gradient boosting','model': GradientBoostingClassifier(), 
     #'param_grid':{'loss':['deviance', 'exponential'], 'learning_rate':[0.01,0.001],
      #            'n_estimators': [100,500,1000], 'max_depth':[3,10,20],
       #            'criterion':['friedman_mse', 'mse','mae'],'min_samples_split':[0.01,0.1,0.2], 
        #           'min_samples_leaf': [0.01,0.1,0.2], 'min_weight_fraction_leaf':[0.01, 0.05],
         #         'subsample':[0.5,0.7,1], 'max_features':['log2','sqrt'],
          #        'max_leaf_nodes':[10, 50, 100]}}
]

## We create a dictionary that will contain results from training and testing our model

grid_results = {name: None for name in selected_categories}

In [ ]:
## For each category of feature, we fit models using gridsearch optimisation
## and ten fold cross-validation and we get the results
for category_name in selected_categories:
    gs = GridSearchCV(params[0]['model'], params[0]['param_grid'], cv = skf, scoring={'F1':'f1'}, refit=False)
    grid_results[category_name] = gs.fit(np.array(list(df[category_name].as_matrix())),
                                                                  df['claim_label'].apply(lambda x: 1 if x else 0).as_matrix())

In [9]:
## For each category of features
## we keep the configuration from gridsearch with the best mean F1 score across folds
## and we look at the mean f-measure score

{k:np.max(v.cv_results_['mean_test_F1']) for k,v in grid_claim_results.items()}

{'claim_lda_0.1_10': 0.5842423420158799,
 'document_lda_10.0_300': 0.589995896522416,
 'document_lsi_6.0_180': 0.530501900349972}